# Is Police providing justice or ......🤐🔫

### Only the powerful person can ask for peace not the weak person. <br>
### Having said that the powerful person should remember his duties and the limit to which it can be extended. <br>
### So lets find whether police is providing justice or ..... <br><br>

### Task  : Find whether police shooting was justified or not . 
### 1. EDA -- on data to find reason for shooting <br> 2. Find areas where shooting like this happended more often  <br> 3. Further analysis.
### Given : data-police-shootings

## Please comment and upvote the notebook if you like it.Thank You🙏😊

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Ploting and visualisations 
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px 
from plotly.offline import download_plotlyjs,init_notebook_mode, iplot
import plotly.tools as tls 
import plotly.figure_factory as ff 
py.init_notebook_mode(connected=True)
# ----------------------- #
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/data-police-shootings/fatal-police-shootings-data.csv')

# Let's Begin

### Let's start by seeing what type of data we have and what it is.

In [ ]:
display(data.info(),data.head())

### Well from primary analysis we can say that these are a lot of shooting by police. Apart from that we have 14 columns and 5416 rows of data.

# 1. Finding Missing Values

### Let's find what all information is missing and what all features needs to be created. We may do Feature Engineering later in the notebook if required. 

In [ ]:
def missing_plot(dataset,key):
    null_feat = pd.DataFrame(len(dataset[key]) - dataset.isnull().sum(), columns = ['Count'])
    percentage_null = pd.DataFrame((dataset.isnull().sum())/len(dataset[key])*100, columns = ['Count'])
    percentage_null = percentage_null.round(2)

    trace = go.Bar(x = null_feat.index, y = null_feat['Count'] ,opacity = 0.8, text = percentage_null['Count'],  textposition = 'auto'
                   ,marker=dict(color = 'darkorange',line=dict(color='#000000',width=1.5)))

    layout = dict(title =  "Missing Values in dataset in percentage%")

    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)


In [ ]:
missing_plot(data,'race')

### These are the missing values in the data.
armed  -- 227 -- 4.19%<br>
age    -- 235 -- 4.34%<br>
gender -- 2   -- 0.04%<br>
race   -- 521 -- 9.62%<br>
flee   -- 249 -- 4.60%<br>


### All the missing values hold very little %age  thus I am going to drop them simply. The main idea is to find if police shooting was justified or justified or not so filling missing data can be kept aside for the time being.

In [ ]:
data.dropna(inplace=True)

# 2. Finding reason for shooting 

### 1. How many deaths occured till now 

### Here we need to do some feature engineering to seperate months and Years as per_day evaluation is difficult.

In [ ]:
data['month'] = pd.to_datetime(data['date']).dt.month
data['year'] = pd.to_datetime(data['date']).dt.year

## Daywise Shooting by police

In [ ]:
daily_shootouts = data[['date']]
daily_shootouts['kills'] = 1
daily_shootouts=daily_shootouts.groupby('date').sum()
daily_shootouts = daily_shootouts.reset_index()

def line_plot(data,var):
    
    trace = go.Scatter(x=data[var],y=data['kills'],line=dict(color='darkorange',width=2))
    
    layout = dict(title='Deaths per {}'.format(var))
    
    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)

In [ ]:
line_plot(daily_shootouts,'date')

## Its not that clear so lets evaluate year.

## Yearly Shooting by police

In [ ]:
daily_shootouts = data[['year']]
daily_shootouts['kills'] = 1
daily_shootouts=daily_shootouts.groupby('year').sum()
daily_shootouts = daily_shootouts.reset_index()

line_plot(daily_shootouts,'year')

## So Shooting by police has been on a decline, from 900 cases in 2015 to 336 cases in 2020.

## Deaths per month per year

In [ ]:
daily_shootouts = data[['year','month']]
daily_shootouts['kills'] = 1
fig = px.bar(daily_shootouts, x='month', y='kills',color_discrete_sequence =['darkorange'], 
                facet_col='year')
fig.show()

### Here we can see deaths per month per year. And found that deaths per months was high in 2015,2018,2019 year.

# Deaths based on Race of the person

#### A - Asian, W - White, H - Hispanic, B - Black, O - Others, N - Native 
#### are the different races whose data we have

In [ ]:
def target_count(data,column,height):
    trace = go.Bar( x = data[column].value_counts().values.tolist(),
    y = data[column].unique(),
    orientation = 'h',
    text = data[column].value_counts().values.tolist(),
    textfont=dict(size=20),
    textposition = 'auto',
    opacity = 0.5,marker=dict(color='darkorange',
            line=dict(color='#000000',width=1.5))
    )
    layout = (dict(title= "EDA of {} column".format(column),
                  autosize=True,height=height,))
    fig = dict(data = [trace], layout=layout)
    
    py.iplot(fig)

# --------------- donut chart to show there percentage -------------------- # 

def target_pie(data,column):
    trace = go.Pie(labels=data[column].unique(),values=data[column].value_counts(),
                  textfont=dict(size=15),
                   opacity = 0.5,marker=dict(
                   colorssrc='tealrose',line=dict(color='#000000', width=1.5)),
                   hole=0.6)
                  
    layout = dict(title="Dounat chart to see %age ")
    fig = dict(data=[trace],layout=layout)
    py.iplot(fig)
    

In [ ]:
target_count(data,'race',400)
target_pie(data,'race')

A - Asian, W - White, H - Hispanic, B - Black, O - Others, N - Native <br>
### Here we can clearly see that half of the people killed were Asian followed by White and Hispanic. 

# Age group to which these people belonged. 

### Let's also see which age group was targeted more and which gender too. 

In [ ]:
def density_plot(var):
    M = data[data['gender'] == "M"] # Data of all the males only 
    F = data[data['gender'] == "F"] # Data of all the females only 
    
    hist_data = [F[var],M[var]]
    labels = ["F","M"]
    color = ['pink','skyblue']
    fig = ff.create_distplot(hist_data,labels,colors = color,show_hist=True,curve_type='kde')
    fig['layout'].update(title = 'Most affected age group with gender wise segregation')
    
    py.iplot(fig, filename = 'Density Plot')

In [ ]:
density_plot('age')

### All age groups have been affected with 18 - 60 age male most affected and 20 - 50 and 53-60 year females were most effected. 

## Manner_of_deaths 

#### Now lets see how did these people were handled by the police.

In [ ]:
target_count(data,'manner_of_death',300)
target_pie(data,'manner_of_death')

## Its quite clear that instead of Tasering 94.8% people where shot down straight-away. 

## But was it required ??? <br> Let's check whether they were armed or not and if they were what was the threat level they had... 

## Armed or not...

In [ ]:
target_count(data,'armed',700)
target_pie(data,'armed')

### From the above illustrations we can see that 59% of the people were having guns but 15% were unarmed and 7.64% were carring tow weapons. So approx. 22.64% were not carring any leathal weapon yet instead of getting simply tasered they were shot. 
### Let's dig deeper and see what kind of threat level they possesed. 

# Areas with most police shooting

In [ ]:
shootout_by_states = data['state'].value_counts()
shootout_by_states = pd.DataFrame(shootout_by_states)
shootout_by_states=shootout_by_states.reset_index()
fig = px.pie(shootout_by_states, values='state', names='index', color_discrete_sequence=px.colors.qualitative.Vivid,hole=0.5)
fig.show()

## The most affected states were CA,TX,FL,AZ,CO,GA etc..

# Threat_level V.S signs_of_mental_illness

### Here we will try to see if the people killed were showing signs_of_mental_illness and what kind of threat they posed. 

### Note : I am taking values of ['gun', 'unarmed', 'toy weapon', 'nail gun', 'knife','shovel']  amd converting rest into 'other' type. 

In [ ]:
daily_shootouts = data[['threat_level','signs_of_mental_illness','armed']]
daily_shootouts['kills'] = 1

daily_shootouts['armed'] = daily_shootouts['armed'].replace(['hammer','hatchet', 'undetermined', 'sword', 'machete', 'box cutter', 'metal object', 'screwdriver', 'lawn mower blade', 'flagpole', 'guns and explosives', 'cordless drill', 'metal pole', 'Taser', 'metal pipe', 'metal hand tool', 'blunt object', 'metal stick', 'sharp object', 'meat cleaver', 'carjack', 'chain', "contractor's level", 'unknown weapon', 'stapler', 'crossbow', 'bean-bag gun', 'baseball bat and fireplace poker', 'straight edge razor', 'gun and knife', 'ax', 'brick', 'baseball bat', 'hand torch', 'chain saw', 'garden tool', 'scissors', 'pole', 'pick-axe', 'flashlight', 'vehicle', 'spear', 'chair', 'pitchfork', 'hatchet and gun', 'rock', 'piece of wood', 'bayonet', 'glass shard', 'motorcycle', 'pepper spray', 'metal rake', 'baton', 'crowbar', 'oar', 'machete and gun', 'air conditioner', 'pole and knife', 'beer bottle', 'pipe', 'baseball bat and bottle', 'fireworks', 'pen', 'chainsaw', 'gun and sword', 'gun and car', 'pellet gun', 'BB gun', 'incendiary device', 'samurai sword', 'bow and arrow', 'gun and vehicle', 'vehicle and gun', 'wrench', 'walking stick', 'barstool', 'grenade', 'BB gun and vehicle', 'wasp spray', 'air pistol', 'baseball bat and knife', 'vehicle and machete', 'ice pick', 'car, knife and mace']
,'other')
fig = px.bar(daily_shootouts, y='armed', x='kills',facet_col='threat_level',facet_row='signs_of_mental_illness',color='armed')
fig.update_traces(marker_color=['rgb(00, 83, 109)'],marker_line_color='darkorange',
                 marker_line_width=1.5, opacity=0.6)
fig.show()


# Final Remarks

## 1. People without mental_illness with threat_level = other and undertermind where killer unnecessarily. Even people who were unarmed and not carring any guns or were with toy weapon were killed unnecessarily. <br>

## 2. People with mental_illness with gun and threat_level = attack can be considered leathal but rest could be tasered instead of killing them.

## 3. In any case people with threat_level -- other and undefined could have been tasered and not shot down immediatly unless they were tring any attack. 

# Please comment and upvote !!🙏😊😊